In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [5]:
df = pd.read_csv('../data/cleaned_for_modeling.csv')

In [6]:
df.keys()

Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME',
       'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5',
       'hour', 'month', 'day_of_week', 'index_right', 'BoroCode', 'BoroName',
       'Shape_Leng', 'Shape_Area', 'Type code 1', 'Type code 1 lower',
       'count vehicles', 'veh_type', 'veh_group', 'num_vehicles', 'hour_bin',
       'cf1_clea

## Create Target(y)
1 -> crash involved >= 1 injured pedestrian

0 -> no pedestrian injuries

In [7]:
# create y predictor 
df['ped_injured'] = (df['NUMBER OF PEDESTRIANS INJURED'] > 0).astype(int)

In [8]:
# sanity check
df['ped_injured'].value_counts()


ped_injured
0    231189
1     22924
Name: count, dtype: int64

In [9]:
df['ped_injured'].value_counts()
df['ped_injured'].value_counts(normalize=True)

ped_injured
0    0.909788
1    0.090212
Name: proportion, dtype: float64

### Understanding the ouput
- dataset is highly imbalanced (91% vs 9%)
- This is normal for injury prediction, but it affects all modeling steps because
    - If I predict 0 for every crash, I'd get 91% accuracy.
    - So accuracy is meaningless here.
    - I have to optimize Recall or F1 for class 1 (injury).
    - I have to compare against a baseline model that respects imbalance.

### Building a baseline model first
- hourly average

In [12]:
X = df[['hour']]

In [10]:
y = df['ped_injured']

In [13]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [14]:
baseline_rate = y_train.mean()
baseline_rate

np.float64(0.09021102857986128)

This means that every real classification model I build next must beat this 9% baseline. If it doesn’t beat the baseline, it’s worthless.

In [ ]:
import numpy as np

y_pred_baseline = np.full(shape=len(y_test), fill_value=baseline_rate)

For every crash, the model predicts a 9% chance a pedestrian is injured

In [16]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae_baseline = mean_absolute_error(y_test, y_pred_baseline)
mse_baseline = mean_squared_error(y_test, y_pred_baseline)

print("Baseline (global average) injury rate:", round(baseline_rate, 4))
print("Baseline MAE:", round(mae_baseline, 4))
print("Baseline MSE:", round(mse_baseline, 4))


Baseline (global average) injury rate: 0.0902
Baseline MAE: 0.1641
Baseline MSE: 0.0821


### MAE AND MSE
- We use regression metrics here because your baseline outputs a probability, not 0/1.

MAE: Mean Absolute Error
    - On average, how far off were our predictions?
    - The baseline is wrong by about 16% on average

MSE: Mean Squared Error
    - How big are the errors, but squared to punish large mistakes?
    - It gives us a target: our real models need to have better (lower) MSE than 0.0821
